In [ ]:
!pip install mne
!pip install mne-icalabel
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.4 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.preprocessing import ICA
from mne_icalabel import label_components
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Constants
FS = 128  # Sampling frequency
METADATA_COLUMNS =  [
    'EEG.Counter', 'EEG.Interpolated', 'EEG.RawCq', 'EEG.Battery',
    'EEG.BatteryPercent', 'EEG.MarkerHardware'
]

print("Libraries imported and constants initialized.")

Libraries imported and constants initialized.


In [ ]:
import os
import re
import pandas as pd

# Function to load and preprocess each participant's EEG CSV individually
def load_dataset_file_by_file(root_folder):
    dataset = []  # List of tuples: (dataframe, label, participant_id)

    for letter_folder in sorted(os.listdir(root_folder)):  # Optional: sort for consistency
        letter_path = os.path.join(root_folder, letter_folder)
        if not os.path.isdir(letter_path):
            print(f"Skipping {letter_path} (not a directory)")
            continue

        print(f"Processing letter folder: {letter_folder}")
        label = letter_folder  # Use folder name as label

        for file in os.listdir(letter_path):
            file_path = os.path.join(letter_path, file)
            if not os.path.isfile(file_path) or not file.endswith('.csv'):
                print(f"  Skipping {file} (not a .csv file)")
                continue

            print(f"  Reading participant file: {file}")
            try:
                df = pd.read_csv(file_path, skiprows=1)

                # Extract participant name using regex
                match = re.match(r'(par\.\d+)', file)
                participant_name = match.group(0) if match else "unknown"

                # Append as a tuple: (dataframe, label, participant_id)
                dataset.append((df, label, participant_name))
            except Exception as e:
                print(f"  Error reading {file_path}: {e}")
                continue

    return dataset

# Example usage
ROOT_FOLDER = "/content/drive/MyDrive/كلمات"
dataset = load_dataset_file_by_file(ROOT_FOLDER)

# Check an example
print(f"\nTotal samples loaded: {len(dataset)}")
print("Example entry:")
df_sample, label_sample, participant_sample = dataset[0]
print(f"Participant: {participant_sample}, Label: {label_sample}")
print(df_sample.head())


Processing letter folder: اختر
  Reading participant file: par.3 اختر_EPOCX_206412_2023.12.24T10.58.33+03.00.md.csv
  Reading participant file: par.20  اختر_EPOCX_206412_2023.12.24T12.49.20+03.00.md.csv
  Reading participant file: par.17 اختر_EPOCX_206412_2023.12.24T13.25.58+03.00.md.csv
  Reading participant file: par.6 اختر_EPOCX_206412_2023.12.21T13.06.47+03.00.md.csv
  Reading participant file: par.11 اختر_EPOCX_206412_2023.12.25T11.16.35+03.00.md.csv
  Reading participant file: par.22 اختر_EPOCX_206412_2023.12.24T14.22.05+03.00.md.csv
  Reading participant file: par.2 اختر_EPOCX_206420_2023.12.25T14.01.11+03.00.md.csv
  Reading participant file: par.14 اختر_EPOCX_206412_2023.12.25T10.47.32+03.00.md.csv
  Reading participant file: par.8 اختر_EPOCX_206412_2023.12.24T11.27.08+03.00.md.csv
  Reading participant file: par.15 اختر_EPOCX_206412_2023.12.24T11.49.28+03.00.md.csv
  Reading participant file: par.7 اختر_EPOCX_206412_2023.12.24T12.18.26+03.00.md.csv
  Reading participant file:

In [ ]:
# Metadata columns to exclude from EEG signal channels
METADATA_COLUMNS = ['Time', 'Event Id', 'Event Date', 'Event Duration', 'Event Description']

# Function to validate EEG data for a single DataFrame
def validate_eeg_data(df):
    # Keep only EEG channels (not metadata)
    valid_columns = [col for col in df.columns if col.startswith('EEG.') and col not in METADATA_COLUMNS]

    if not valid_columns:
        print("  ❌ No valid EEG signal columns identified. Skipping this DataFrame.")
        return None, None

    print(f"  ✅ Validating data with shape: {df.shape}")
    return df[valid_columns].copy(), valid_columns

# Function to validate all entries in the dataset
def validate_all_eeg_data(dataset):
    validated_dataset = []           # Will store (cleaned_df, label, participant)
    valid_columns_map = {}           # Optional: track valid columns per participant
    skipped = 0

    for idx, (df, label, participant) in enumerate(dataset):
        print(f"\n🔍 Processing sample {idx + 1}/{len(dataset)} - Participant: {participant}, Label: {label}")
        validated_df, valid_columns = validate_eeg_data(df)

        if validated_df is not None:
            validated_dataset.append((validated_df, label, participant))
            valid_columns_map[participant] = valid_columns
        else:
            print(f"  ⚠️ Skipping sample from {participant}")
            skipped += 1

    print(f"\n✅ EEG validation complete. Valid samples: {len(validated_dataset)}, Skipped: {skipped}")
    return validated_dataset, valid_columns_map

# Run validation
validated_dataset, valid_columns_map = validate_all_eeg_data(dataset)

# Optional: Preview
df_example, label_example, participant_example = validated_dataset[0]
print(f"\n👤 Participant: {participant_example}, 🏷️ Label: {label_example}, 📐 EEG shape: {df_example.shape}")



🔍 Processing sample 1/359 - Participant: par.3, Label: اختر
  ✅ Validating data with shape: (1386, 68)

🔍 Processing sample 2/359 - Participant: par.20, Label: اختر
  ✅ Validating data with shape: (1413, 68)

🔍 Processing sample 3/359 - Participant: par.17, Label: اختر
  ✅ Validating data with shape: (1388, 68)

🔍 Processing sample 4/359 - Participant: par.6, Label: اختر
  ✅ Validating data with shape: (1394, 68)

🔍 Processing sample 5/359 - Participant: par.11, Label: اختر
  ✅ Validating data with shape: (1369, 68)

🔍 Processing sample 6/359 - Participant: par.22, Label: اختر
  ✅ Validating data with shape: (1379, 68)

🔍 Processing sample 7/359 - Participant: par.2, Label: اختر
  ✅ Validating data with shape: (1380, 68)

🔍 Processing sample 8/359 - Participant: par.14, Label: اختر
  ✅ Validating data with shape: (1384, 68)

🔍 Processing sample 9/359 - Participant: par.8, Label: اختر
  ✅ Validating data with shape: (1376, 68)

🔍 Processing sample 10/359 - Participant: par.15, Label: ا

In [ ]:
CHANNELS_TO_REMOVE = ['EEG.T8', 'EEG.O1', 'EEG.F3']
DESIRED_EEG_CHANNELS = [
    'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5', 'EEG.T7', 'EEG.P7',
    'EEG.O1', 'EEG.O2', 'EEG.P8', 'EEG.T8', 'EEG.FC6', 'EEG.F4', 'EEG.F8', 'EEG.AF4'
]

def clean_validated_dataset(validated_dataset):
    cleaned_dataset = []

    for idx, (df, label, participant) in enumerate(validated_dataset):
        print(f"\n🧹 Cleaning sample {idx + 1}/{len(validated_dataset)} - Participant: {participant}, Label: {label}")

        # Select only desired channels first
        available_channels = [ch for ch in DESIRED_EEG_CHANNELS if ch in df.columns]
        df_cleaned = df[available_channels].copy()

        # Drop the undesired ones explicitly
        removed_channels = [ch for ch in CHANNELS_TO_REMOVE if ch in df_cleaned.columns]
        df_cleaned.drop(columns=removed_channels, inplace=True)

        print(f"  Removed channels: {removed_channels}")
        print(f"  Shape after cleaning: {df_cleaned.shape}")

        cleaned_dataset.append((df_cleaned, label, participant))

    print(f"\n✅ Finished cleaning all EEG samples. Total cleaned samples: {len(cleaned_dataset)}")
    return cleaned_dataset

# Run the cleaning step
cleaned_dataset = clean_validated_dataset(validated_dataset)

# Example check
df_clean, label_clean, participant_clean = cleaned_dataset[0]
print(f"\n👤 Participant: {participant_clean}, 🏷️ Label: {label_clean}, 📐 Cleaned EEG shape: {df_clean.shape}")



🧹 Cleaning sample 1/359 - Participant: par.3, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cleaning: (1386, 11)

🧹 Cleaning sample 2/359 - Participant: par.20, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cleaning: (1413, 11)

🧹 Cleaning sample 3/359 - Participant: par.17, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cleaning: (1388, 11)

🧹 Cleaning sample 4/359 - Participant: par.6, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cleaning: (1394, 11)

🧹 Cleaning sample 5/359 - Participant: par.11, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cleaning: (1369, 11)

🧹 Cleaning sample 6/359 - Participant: par.22, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cleaning: (1379, 11)

🧹 Cleaning sample 7/359 - Participant: par.2, Label: اختر
  Removed channels: ['EEG.T8', 'EEG.O1', 'EEG.F3']
  Shape after cl

In [ ]:
import mne

FS = 128  # Set your actual sampling frequency here

# Function to create an MNE Raw object
def create_raw_object(eeg_data, sfreq=FS):
    data = eeg_data.to_numpy().T  # Transpose to shape (n_channels, n_times)
    ch_names = eeg_data.columns.tolist()
    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=['eeg'] * len(ch_names))
    raw = mne.io.RawArray(data, info, verbose=False)
    return raw

# Function to rename EEG channels for standard montage compatibility
def rename_eeg_channels(raw_object):
    rename_dict = {
        'EEG.AF3': 'AF3', 'EEG.FC6': 'FC6', 'EEG.AF4': 'AF4', 'EEG.F7': 'F7',
        'EEG.FC5': 'FC5', 'EEG.T7': 'T7', 'EEG.O2': 'O2',
        'EEG.P7': 'P7', 'EEG.P8': 'P8',
        'EEG.F4': 'F4', 'EEG.F8': 'F8'
    }


    raw_object.rename_channels(rename_dict)
    return raw_object

# Create Raw objects for each cleaned entry (per participant)
raw_dataset = []  # Will store (raw_object, label, participant)

for idx, (df_clean, label, participant) in enumerate(cleaned_dataset):
    print(f"\n🧠 Creating Raw object {idx + 1}/{len(cleaned_dataset)} - Participant: {participant}, Label: {label}")

    try:
        raw = create_raw_object(df_clean)
        raw = rename_eeg_channels(raw)
        raw_dataset.append((raw, label, participant))
        print("  ✅ Successfully created and renamed Raw object.")
    except Exception as e:
        print(f"  ❌ Failed to create Raw object for {participant}, label {label}: {e}")



🧠 Creating Raw object 1/359 - Participant: par.3, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 2/359 - Participant: par.20, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 3/359 - Participant: par.17, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 4/359 - Participant: par.6, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 5/359 - Participant: par.11, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 6/359 - Participant: par.22, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 7/359 - Participant: par.2, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 8/359 - Participant: par.14, Label: اختر
  ✅ Successfully created and renamed Raw object.

🧠 Creating Raw object 9/359 - Participant: par.8, Label: اختر
  ✅ Successfully created and renamed

In [ ]:
from mne.preprocessing import ICA
from mne_icalabel import label_components
import numpy as np

# Apply filters and referencing to a Raw object
def apply_filters_and_reference(raw):
    raw.notch_filter(freqs=50.0, verbose=False)
    raw.filter(l_freq=1.0, h_freq=50.0, verbose=False)
    raw.set_eeg_reference('average', projection=True, verbose=False)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, verbose=False)
    print("  ✅ Filters and referencing applied.")
    return raw

# Perform ICA and clean artifacts for a Raw object
def perform_ica(raw, brain_threshold=0.5):
    ica = ICA(method='infomax', fit_params={'extended': True}, n_components=11, max_iter=1000, random_state=42)
    ica.fit(raw, verbose=False)

    ic_labels = label_components(raw, ica, method="iclabel")

    non_brain_components = [
        idx for idx, (label, prob) in enumerate(zip(ic_labels['labels'], ic_labels['y_pred_proba']))
        if label != 'brain' or prob < brain_threshold
    ]

    ica.exclude = non_brain_components
    raw_cleaned = ica.apply(raw.copy(), verbose=False)

    print(f"  ✅ ICA completed. Excluded components: {non_brain_components}")
    return raw_cleaned

# Check and clean NaN/Inf values from Raw data
def clean_nan_inf(raw):
    data = raw.get_data()
    if np.isnan(data).any() or np.isinf(data).any():
        print("  ⚠️ Detected NaN or Inf values.")
        for i in range(data.shape[0]):
            channel_data = data[i, :]
            channel_mean = np.nanmean(channel_data)
            channel_data[np.isnan(channel_data)] = channel_mean
            channel_data[np.isinf(channel_data)] = channel_mean
            data[i, :] = channel_data
        print("  ✅ NaN/Inf values replaced with channel means.")
    else:
        print("  ✅ No NaN or Inf values detected.")
    raw._data = data
    return raw

# Final cleaned raw dataset: list of (raw_object, label, participant)
cleaned_raw_dataset = []

for idx, (raw_object, label, participant) in enumerate(raw_dataset):
    print(f"\n🧪 Processing sample {idx + 1}/{len(raw_dataset)} - Participant: {participant}, Label: {label}")
    try:
        raw = apply_filters_and_reference(raw_object)
        raw = clean_nan_inf(raw)
        raw = perform_ica(raw)
        cleaned_raw_dataset.append((raw, label, participant))
        print(f"  ✅ Successfully processed sample for {participant} ({label})")
    except Exception as e:
        print(f"  ❌ Failed to process sample for {participant} ({label}): {e}")



🧪 Processing sample 1/359 - Participant: par.3, Label: اختر
  ✅ Filters and referencing applied.
  ✅ No NaN or Inf values detected.


<ipython-input-28-4a2ee7b92820>:9: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference('average', projection=True, verbose=False)
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


  ✅ ICA completed. Excluded components: [1, 2, 4, 5, 7, 9]
  ✅ Successfully processed sample for par.3 (اختر)

🧪 Processing sample 2/359 - Participant: par.20, Label: اختر
  ✅ Filters and referencing applied.
  ✅ No NaN or Inf values detected.


<ipython-input-28-4a2ee7b92820>:9: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference('average', projection=True, verbose=False)
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


  ✅ ICA completed. Excluded components: [0, 6, 7, 10]
  ✅ Successfully processed sample for par.20 (اختر)

🧪 Processing sample 3/359 - Participant: par.17, Label: اختر
  ✅ Filters and referencing applied.
  ✅ No NaN or Inf values detected.


...

  ✅ ICA completed. Excluded components: [0]
  ✅ Successfully processed sample for par.23 (يمين)

🧪 Processing sample 358/359 - Participant: par.16, Label: يمين
  ✅ Filters and referencing applied.
  ✅ No NaN or Inf values detected.


<ipython-input-28-4a2ee7b92820>:9: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference('average', projection=True, verbose=False)
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


  ✅ ICA completed. Excluded components: [2, 5, 7, 8, 9]
  ✅ Successfully processed sample for par.16 (يمين)

🧪 Processing sample 359/359 - Participant: par.15, Label: يمين
  ✅ Filters and referencing applied.
  ✅ No NaN or Inf values detected.


<ipython-input-28-4a2ee7b92820>:9: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference('average', projection=True, verbose=False)
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")
<ipython-input-28-4a2ee7b92820>:20: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(raw, ica, method="iclabel")


  ✅ ICA completed. Excluded components: [0, 5, 7, 8, 10]
  ✅ Successfully processed sample for par.15 (يمين)


In [ ]:
# Interpolation and NaN handling for Raw objects after ICA and filtering
def interpolate_and_handle_nans(raw_cleaned):
    try:
        # Step 1: Interpolate bad channels (if any)
        if raw_cleaned.info['bads']:
            print(f"  ⚠️ Interpolating bad channels: {raw_cleaned.info['bads']}")
            raw_cleaned.interpolate_bads(reset_bads=True)
        else:
            print("  ✅ No bad channels to interpolate.")

        # Step 2: Extract EEG data into DataFrame
        eeg_data = pd.DataFrame(raw_cleaned.get_data().T, columns=raw_cleaned.ch_names)

        # Step 3: Interpolate NaN values in the EEG data
        if eeg_data.isna().sum().sum() > 0:
            print("  ⚠️ NaN values found. Performing interpolation...")
            eeg_data = eeg_data.interpolate(method='linear', axis=0)
            eeg_data = eeg_data.fillna(method='ffill').fillna(method='bfill')
        else:
            print("  ✅ No NaN values detected in EEG data.")

        # Step 4: Recreate Raw object with cleaned data
        raw_cleaned = mne.io.RawArray(eeg_data.T.to_numpy(), raw_cleaned.info)
        return raw_cleaned

    except Exception as e:
        print(f"  ❌ Error during interpolation and NaN handling: {e}")
        return raw_cleaned


# Final list of fully cleaned raw samples (after ICA + interpolation)
fully_cleaned_raw_dataset = []

for idx, (raw_object, label, participant) in enumerate(cleaned_raw_dataset):
    print(f"\n🧼 Interpolating + NaN Cleaning sample {idx + 1}/{len(cleaned_raw_dataset)} - Participant: {participant}, Label: {label}")

    try:
        cleaned_raw = interpolate_and_handle_nans(raw_object)
        fully_cleaned_raw_dataset.append((cleaned_raw, label, participant))
        print(f"  ✅ Successfully cleaned sample for {participant} ({label})")

    except Exception as e:
        print(f"  ❌ Failed to process sample for {participant} ({label}): {e}")



🧼 Interpolating + NaN Cleaning sample 1/359 - Participant: par.3, Label: اختر
  ✅ No bad channels to interpolate.
  ✅ No NaN values detected in EEG data.
Creating RawArray with float64 data, n_channels=11, n_times=1386
    Range : 0 ... 1385 =      0.000 ...    10.820 secs
Ready.
  ✅ Successfully cleaned sample for par.3 (اختر)

🧼 Interpolating + NaN Cleaning sample 2/359 - Participant: par.20, Label: اختر
  ✅ No bad channels to interpolate.
  ✅ No NaN values detected in EEG data.
Creating RawArray with float64 data, n_channels=11, n_times=1413
    Range : 0 ... 1412 =      0.000 ...    11.031 secs
Ready.
  ✅ Successfully cleaned sample for par.20 (اختر)

🧼 Interpolating + NaN Cleaning sample 3/359 - Participant: par.17, Label: اختر
  ✅ No bad channels to interpolate.
  ✅ No NaN values detected in EEG data.
Creating RawArray with float64 data, n_channels=11, n_times=1388
    Range : 0 ... 1387 =      0.000 ...    10.836 secs
Ready.
  ✅ Successfully cleaned sample for par.17 (اختر)

🧼 

In [ ]:
# Function to apply Z-score normalization to EEG data
def z_score_normalize_eeg_data(raw):
    """
    Normalize the EEG data in the Raw object using Z-score normalization
    (zero mean, unit variance) per channel.
    """
    eeg_data = raw.get_data()
    mean = np.mean(eeg_data, axis=1, keepdims=True)
    std = np.std(eeg_data, axis=1, keepdims=True)

    # Avoid division by zero
    std[std == 0] = 1e-10

    normalized_data = (eeg_data - mean) / std
    raw._data = normalized_data
    print("  ✅ Applied Z-score normalization.")
    return raw

# Final dataset with Z-score normalization applied
z_score_normalized_dataset = []

for idx, (raw_object, label, participant) in enumerate(fully_cleaned_raw_dataset):
    print(f"\n🔄 Normalizing sample {idx + 1}/{len(fully_cleaned_raw_dataset)} - Participant: {participant}, Label: {label}")

    try:
        normalized_raw = z_score_normalize_eeg_data(raw_object)
        z_score_normalized_dataset.append((normalized_raw, label, participant))
        print(f"  ✅ Normalized sample for {participant} ({label})")

    except Exception as e:
        print(f"  ❌ Failed to normalize sample for {participant} ({label}): {e}")



🔄 Normalizing sample 1/359 - Participant: par.3, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.3 (اختر)

🔄 Normalizing sample 2/359 - Participant: par.20, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.20 (اختر)

🔄 Normalizing sample 3/359 - Participant: par.17, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.17 (اختر)

🔄 Normalizing sample 4/359 - Participant: par.6, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.6 (اختر)

🔄 Normalizing sample 5/359 - Participant: par.11, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.11 (اختر)

🔄 Normalizing sample 6/359 - Participant: par.22, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.22 (اختر)

🔄 Normalizing sample 7/359 - Participant: par.2, Label: اختر
  ✅ Applied Z-score normalization.
  ✅ Normalized sample for par.2 (اختر)

🔄 Normalizing sample 8/359 - Participan

In [ ]:
import numpy as np
import pandas as pd

# Set segment length (e.g., 1200 samples = 9.375 seconds if FS = 128)
segment_length = 1200

# Final dataset: list of (segment_array, label, participant)
segmented_dataset = []

# Process each sample
for idx, (raw_object, label, participant) in enumerate(z_score_normalized_dataset):
    print(f"\n🔄 Segmenting sample {idx + 1}/{len(z_score_normalized_dataset)} - Participant: {participant}, Label: {label}")

    try:
        data = raw_object.get_data().T  # Shape: (time, channels)
        num_segments = len(data) // segment_length

        for i in range(num_segments):
            segment = data[i * segment_length: (i + 1) * segment_length]  # (segment_length, channels)
            segmented_dataset.append((segment, label, participant))

        if len(data) % segment_length != 0:
            partial_len = len(data) % segment_length
            print(f"  ℹ️ Partial segment ({partial_len} samples) ignored for {participant} ({label})")

        print(f"  ✅ {num_segments} full segments created for {participant} ({label})")

    except Exception as e:
        print(f"  ❌ Failed to segment sample for {participant} ({label}): {e}")


# 🔽 Convert to flat DataFrame for CSV export
flat_data = []
for segment_array, label, participant in segmented_dataset:
    flat_segment = segment_array.flatten()  # shape: (segment_length * channels,)
    row = list(flat_segment) + [label, participant]
    flat_data.append(row)

# Generate column names
num_features = segment_array.shape[0] * segment_array.shape[1]  # time * channels
column_names = [f"feature_{i}" for i in range(num_features)] + ["label", "participant"]

# Create DataFrame
df = pd.DataFrame(flat_data, columns=column_names)

# 💾 Save to CSV
csv_filename = "segmented_letter_data.csv"
df.to_csv(csv_filename, index=False)
print(f"\n✅ Segmented letter data saved to '{csv_filename}'")



🔄 Segmenting sample 1/359 - Participant: par.3, Label: اختر
  ℹ️ Partial segment (186 samples) ignored for par.3 (اختر)
  ✅ 1 full segments created for par.3 (اختر)

🔄 Segmenting sample 2/359 - Participant: par.20, Label: اختر
  ℹ️ Partial segment (213 samples) ignored for par.20 (اختر)
  ✅ 1 full segments created for par.20 (اختر)

🔄 Segmenting sample 3/359 - Participant: par.17, Label: اختر
  ℹ️ Partial segment (188 samples) ignored for par.17 (اختر)
  ✅ 1 full segments created for par.17 (اختر)

🔄 Segmenting sample 4/359 - Participant: par.6, Label: اختر
  ℹ️ Partial segment (194 samples) ignored for par.6 (اختر)
  ✅ 1 full segments created for par.6 (اختر)

🔄 Segmenting sample 5/359 - Participant: par.11, Label: اختر
  ℹ️ Partial segment (169 samples) ignored for par.11 (اختر)
  ✅ 1 full segments created for par.11 (اختر)

🔄 Segmenting sample 6/359 - Participant: par.22, Label: اختر
  ℹ️ Partial segment (179 samples) ignored for par.22 (اختر)
  ✅ 1 full segments created for par.22